# Load model and prepare data!

In [1]:
from utils import *

1SCL_A
1RNK_A
1RHT_A
1HLX_A
1HMH_E
1RNG_A
1MME_D
1KAJ_A
1SLO_A
1BIV_A
1ANR_A
1ZIG_A
1ZIH_A
1ETF_A
1ZIF_A
1KPD_A
1IKD_A
1ZDI_S
1AFX_A
1EBQ_A
1EBR_A
1ULL_A
1KIS_B
1KIS_A
1ATO_A
1TLR_A
1VOP_A
1AQO_A
1ATV_A
1ATW_A
1UUU_A
1AUD_B
2U2A_A
1A4T_A
1A60_A
1A51_A
2A9L_A
1A1T_B
1A9N_Q
3PHP_A
2TPK_A
7MSF_S
5MSF_S
1LDZ_A
1ZDK_S
1BVJ_A
1B36_A
1HVU_I
1BGZ_A
2BJ2_B
2BJ2_A
28SP_A
1QFQ_A
17RA_A
1BAU_B
1BZ2_A
1CQ5_A
1QC8_A
484D_B
1D6K_B
1EIY_C
1ESH_A
1EXY_A
1D0U_A
1ESY_A
1EUQ_B
1F9L_A
1FFK_9
1EKZ_B
1F5U_B
1F6Z_A
1F6X_A
1F85_A
1F84_A
1FOQ_A
1FJE_A
1E4P_A
1FQZ_A
1G70_A
1E7K_D
1FYO_A
1FHK_A
1I3X_A
1I4C_A
1I46_A
1IBM_Y
1HWQ_A
1IK1_A
1E95_A
1HS2_A
1JO7_A
1IDV_A
1K5I_A
1K9W_A
1JTW_A
1K6G_A
1K4B_A
1K4A_A
1K6H_A
1JUR_A
1KKS_A
1JWC_A
1L1C_C
1KP7_A
1JOX_A
1K2G_A
1L1W_A
1LC6_A
1LS2_B
1KKA_A
1MFY_A
1M5L_A
1MFJ_A
1MFK_A
1N34_A
1JTJ_A
1MT4_A
1MNX_A
1NA2_A
1NC0_A
1N8X_A
1OQ0_A
1OW9_A
1OSW_A
1M82_A
1NYB_B
1PJY_A
1P6V_B
1P6V_D
1N66_A
1HS1_A
1HS8_A
1HS4_A
1HS3_A
1JZC_A
1QZC_C
1P5M_A
1R2W_C
1P5N_A
1QZC_B
1P5P_A
1QZA_B
1QZB_B

# Prepare data

In [4]:
val_seq = all_seq_ids("val")
print(len(val_seq))

model = load_model()

SyntaxError: invalid syntax (1076635656.py, line 5)

In [12]:
# read the csv file
import pandas as pd

from rhofold.utils.alphabet import get_features


train_list = all_seq_ids(train_csv)
print(train_list)
seq_id = train_list[0]
outputs = seq_id_to_data(seq_id)
print(len(outputs))
print(outputs[0].keys())

['1SCL_A', '1RNK_A', '1RHT_A', '1HLX_A', '1HMH_E', '1RNG_A', '1MME_D', '1KAJ_A', '1SLO_A', '1BIV_A', '1ANR_A', '1ZIG_A', '1ZIH_A', '1ETF_A', '1ZIF_A', '1KPD_A', '1IKD_A', '1ZDI_S', '1AFX_A', '1EBQ_A', '1EBR_A', '1ULL_A', '1KIS_B', '1KIS_A', '1ATO_A', '1TLR_A', '1VOP_A', '1AQO_A', '1ATV_A', '1ATW_A', '1UUU_A', '1AUD_B', '2U2A_A', '1A4T_A', '1A60_A', '1A51_A', '2A9L_A', '1A1T_B', '1A9N_Q', '3PHP_A', '2TPK_A', '7MSF_S', '5MSF_S', '1LDZ_A', '1ZDK_S', '1BVJ_A', '1B36_A', '1HVU_I', '1BGZ_A', '2BJ2_B', '2BJ2_A', '28SP_A', '1QFQ_A', '17RA_A', '1BAU_B', '1BZ2_A', '1CQ5_A', '1QC8_A', '484D_B', '1D6K_B', '1EIY_C', '1ESH_A', '1EXY_A', '1D0U_A', '1ESY_A', '1EUQ_B', '1F9L_A', '1FFK_9', '1EKZ_B', '1F5U_B', '1F6Z_A', '1F6X_A', '1F85_A', '1F84_A', '1FOQ_A', '1FJE_A', '1E4P_A', '1FQZ_A', '1G70_A', '1E7K_D', '1FYO_A', '1FHK_A', '1I3X_A', '1I4C_A', '1I46_A', '1IBM_Y', '1HWQ_A', '1IK1_A', '1E95_A', '1HS2_A', '1JO7_A', '1IDV_A', '1K5I_A', '1K9W_A', '1JTW_A', '1K6G_A', '1K4B_A', '1K4A_A', '1K6H_A', '1JUR_A',

# Example Evaluation

In [5]:
def seq_id_to_data(model, seq_id):
    with torch.inference_mode():
        features = feats(seq_id)
        device = "cuda"
        outputs = model(
            tokens=features["tokens"].to(device),
            rna_fm_tokens=features["rna_fm_tokens"].to(device),
            seq=features["seq"],
        )
        return outputs

In [7]:


with torch.inference_mode():
    seq_id = train_list[3]
    outputs = seq_id_to_data(seq_id)[1]["cords_c1'"][0][0]
    print(eval_tm(seq_id, outputs))

0.003528591711074114


In [8]:
test_list = all_seq_ids(test_csv)
val_list = all_seq_ids(val_csv)
print(len(test_list))
print(len(val_list))

844
844


In [15]:
for idx, seq in enumerate(test_list):
    ft = feats(seq)
    print(idx, ft["seq"], ft["tokens"].shape, ft["rna_fm_tokens"].shape)


0 GGGUGCUCAGUACGAGAGGAACCGCACCC torch.Size([1, 128, 29]) torch.Size([1, 29])
1 GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU torch.Size([1, 10, 34]) torch.Size([1, 34])
2 GGGACUGACGAUCACGCAGUCUAU torch.Size([1, 12, 24]) torch.Size([1, 24])
3 GGGAUAACUUCGGUUGUCCC torch.Size([1, 11, 20]) torch.Size([1, 20])
4 GGCGACCCUGAUGAGGCCGAAAGGCCGAAACCGU torch.Size([1, 128, 34]) torch.Size([1, 34])
5 GGCGCUUGCGUC torch.Size([1, 1, 12]) torch.Size([1, 12])
6 GGCCGAAACUCGUAAGAGUCACCAC torch.Size([1, 2, 25]) torch.Size([1, 25])
7 GGCGCAGUGGGCUAGCGCCACUCAAAAGCCCG torch.Size([1, 64, 32]) torch.Size([1, 32])
8 UUACCCAAGUUUGAGGUAA torch.Size([1, 26, 19]) torch.Size([1, 19])
9 GGCUCGUGUAGCUCAUUAGCUCCGAGCC torch.Size([1, 7, 28]) torch.Size([1, 28])
10 GGCAGAUCUGAGCCUGGGAGCUCUCUGCC torch.Size([1, 80, 29]) torch.Size([1, 29])
11 GGGCGAGAGCCU torch.Size([1, 2, 12]) torch.Size([1, 12])
12 GGGCGCAAGCCU torch.Size([1, 1, 12]) torch.Size([1, 12])
13 GGUCUGGGCGCAGCGCAAGCUGACGGUACAGGCC torch.Size([1, 57, 34]) torch.Size([1, 34

In [16]:
def test_dict_to_submission(test_dict):
    '''
    test_dict: {seq_id: [pred_cords * 5]}
    '''
    rows = []
    for seq_id, preds in test_dict.items():
        sequence = id_to_seq[seq_id]  # Get sequence from previously created id_to_seq dict
        for i, residue in enumerate(sequence, 1):  # 1-based indexing for residue IDs
            row = {
                'ID': f"{seq_id}_{i}",
                'resname': residue,
                'resid': i
            }
            
            # Add coordinates for each of the 5 predictions
            for model_idx, pred in enumerate(preds, 1):
                coords = pred[i-1]  # Get coordinates for this residue (0-based indexing)
                row[f'x_{model_idx}'] = coords[0].item()
                row[f'y_{model_idx}'] = coords[1].item()
                row[f'z_{model_idx}'] = coords[2].item()
            
            rows.append(row)
    
    # Create DataFrame and save to CSV
    submission_df = pd.DataFrame(rows)
    submission_df.to_csv('submission.csv', index=False)
    return submission_df

def eval_val_score(val_dict):
    mean_score = 0
    for seq_id, preds in val_dict.items():
        max_score = 0
        for pred in preds:
            score = eval_tm(seq_id, pred)
            if score > max_score:
                max_score = score
        mean_score += max_score
    mean_score /= len(val_dict)
    return mean_score



In [22]:
import tqdm
def baseline(test_list, max_seq_len=512, n_samples=5):
    results = dict()
    for seq_id in tqdm.tqdm(test_list):
        seq = id_to_seq[seq_id]
        if len(seq) > max_seq_len:
            preds = []
            # TODO: here, let's crop it; now as a placeholder, just put in all zeros.
            for i in range(n_samples):
                preds.append(torch.zeros((len(seq), 3), device=device))
            results[seq_id] = preds
            continue
        outputs = seq_id_to_data(seq_id)
        preds = []
        for i in range(n_samples):
            preds.append(outputs[i]["cords_c1'"][0][0])
        results[seq_id] = preds
    return results

val_dict = baseline(val_list[:20])
print(eval_val_score(val_dict))

100%|██████████| 20/20 [00:13<00:00,  1.51it/s]

0.03798852391191758


In [23]:
results = baseline(test_list)
test_dict_to_submission(results)

100%|██████████| 844/844 [12:01<00:00,  1.17it/s]


,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,1SCL_A_1,G,1,0.880592,-14.967862,14.847087,1.103798,-14.600047,15.202257,1.309041,-14.545917,15.411874,1.501110,-14.577082,15.551977,1.600115,-14.564169,15.760962
1,1SCL_A_2,G,2,5.723164,-12.441339,13.703876,5.816087,-12.255725,13.697553,5.993453,-12.252779,13.770735,6.171873,-12.288349,13.871452,6.268919,-12.292291,14.057161
2,1SCL_A_3,G,3,9.856880,-10.505846,10.350113,9.773094,-10.530338,10.092791,9.892091,-10.598871,10.073586,10.055022,-10.656443,10.149103,10.154890,-10.701406,10.321133
3,1SCL_A_4,U,4,11.789226,-9.293490,5.499051,11.469459,-9.444762,5.105185,11.504141,-9.574741,5.047934,11.646362,-9.664747,5.110266,11.746153,-9.763164,5.270353
4,1SCL_A_5,G,5,11.272184,-8.234337,0.302625,10.739121,-8.512379,-0.070449,10.683502,-8.690971,-0.111415,10.802099,-8.823223,-0.048106,10.900265,-8.975779,0.107624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137090,8Z1F_T_82,U,82,6.302320,-59.509109,12.169044,5.100036,-62.280701,8.302595,4.483331,-63.202713,6.805640,3.714312,-63.845802,6.222885,3.271701,-64.184502,6.386507
137091,8Z1F_T_83,C,83,4.466007,-61.085926,16.982029,0.778647,-64.072426,10.975146,0.004170,-64.910255,9.342413,-0.835030,-65.544281,8.627932,-1.329771,-65.893517,8.664008
137092,8Z1F_T_84,A,84,1.565706,-64.146820,20.260279,-3.618638,-67.134888,11.542344,-4.630943,-67.754700,9.613880,-5.601504,-68.259064,8.668365,-6.147414,-68.564438,8.532651
137093,8Z1F_T_85,U,85,-1.511352,-68.321465,21.853348,-7.142821,-71.178177,10.387381,-8.389748,-71.497452,8.082346,-9.457425,-71.822777,6.915578,-9.997579,-72.062759,6.634630
